# LAB 3-Online_shopper_Call backs_cm_Regularization_Optimization

## Objectives of the LAB:
1. Introduce a variety of call backs, plotting, file and history saving, viewing weights of layers and performance metrics.
2. Perform regularization and Batch normalization.
3. Evaluate performace of the models.
4. Compare performance of the models

## 1. Importing Libraries and mounting drive

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow import keras
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 2. Loading and Preparing Dataset

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/nopynospy/apu_deep_learning/main/online%20shop_train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/nopynospy/apu_deep_learning/main/online%20shop_test.csv')

In [3]:
train_df.shape

(16626, 13)

In [4]:
test_df.shape

(2466, 13)

In [5]:
# Split dataset to independent (X) and target (y) variables
X_train = train_df.iloc[:,:-1]
y_train = train_df.iloc[:,-1]
X_test = test_df.iloc[:,:-1]
y_test = test_df.iloc[:,-1]
# One-hot encoding the target variables from the training, validation and test datasets for classification
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=None, dtype="int")
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=None, dtype="int")
# Clearing the nodes left behind in the previous session to free up memory and preventing slowdown.
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# 4. Misc features
## 4.1 Layer Weights

In [7]:
def create_baseline():
    #Initializing Neural Network
    classifier = Sequential()

    ##Defining the architecture of ANN
   
    # First layer 
    classifier.add(Dense(units = 512, kernel_initializer = 'he_uniform', activation = 'relu', input_dim = 12))
    
    # Second layer
    classifier.add(Dense(units= 128 , kernel_initializer = 'he_uniform', activation = 'relu'))
    
    classifier.add(Dense(units= 96, kernel_initializer = 'he_uniform', activation = 'relu'))
    # Adding the output layer
    classifier.add(Dense(units = 2, kernel_initializer = 'he_uniform', activation = 'softmax'))
    
   
    # Compiling Neural Network
    ## specifying optimizer 
    ## This is the metaparameter, specfiying the metaparameter , (adam is popular optimizer like SGD, and GD, 
    ## the binary_corssentropy is because it has 2 class)
    
    
    # classifier.compile(optimizer = 'SGD', loss = 'binary_crossentropy', metrics = ['accuracy'])
    classifier.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

model_baseline = create_baseline()
print(model_baseline.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               6656      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 96)                12384     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 194       
Total params: 84,898
Trainable params: 84,898
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
hidden1 = model_baseline.layers[1]
weights, biases = hidden1.get_weights()
weights

array([[ 0.04067964, -0.00336043,  0.09331302, ...,  0.00245792,
         0.09733552, -0.08533823],
       [ 0.06069423,  0.09338554,  0.09013639, ..., -0.05562919,
        -0.09915592, -0.07003643],
       [-0.03871035,  0.06407639, -0.04704274, ...,  0.08028995,
        -0.03738622,  0.0967179 ],
       ...,
       [-0.09992407,  0.05533648,  0.05717779, ...,  0.03523437,
        -0.00272727, -0.05270342],
       [-0.07520302, -0.04225121, -0.03619627, ..., -0.06922771,
        -0.03622319, -0.00147122],
       [-0.03755522, -0.06533077, -0.0409788 , ...,  0.05789558,
        -0.04246469,  0.01773887]], dtype=float32)

Save the entire model to a HDF5 file.

The '.h5' extension indicates that the model should be saved to HDF5.

(In this case, to not be only limited to Google Colab and drive, I downloaded it)

In [13]:
model_baseline.save('lab3_baseline.h5')

from google.colab import files
files.download('lab3_baseline.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 4.3 Restoring saved model and history

(Behind the scenes, I uploaded the downloaded h5 file to my GitHub)

In [17]:
!git clone https://github.com/nopynospy/apu_deep_learning.git

Cloning into 'apu_deep_learning'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 65 (delta 27), reused 25 (delta 4), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [18]:
!cd apu_deep_learning/
!ls

apu_deep_learning  lab3_baseline.h5  sample_data


In [19]:
model = keras.models.load_model('lab3_baseline.h5')